### requirements

In [6]:
import torch

### all import tests

In [1]:
# utils
from mltools.utils import cuda_tools

# single file modules
from mltools.distributions import DiagonalGaussianDistribution
from mltools.losses import MultiScaleMSE

# networks
from mltools.networks.blocks import AttnBlock, ResNetBlock, ResNetDown, ResNetUp
from mltools.networks.network_tools import zero_init, get_conv, get_timestep_embedding
from mltools.networks.networks import CUNet, Encoder, Decoder

# models
from mltools.models.model_tools import kl_std_normal, FixedLinearSchedule, SigmoidSchedule, LearnedLinearSchedule, NNSchedule
from mltools.models.vae_model import AutoencoderKL
from mltools.models.vdm_model import VDM, LightVDM

# h5
from mltools.h5 import h5_tools

# connectomics
from mltools.connectomics import ConnectomicsDataset
from mltools.connectomics import connectomics_tools

/n/home12/cfpark00/venv1/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: rch is an invalid version and will not be supported in a future release
  warnings.warn(
/n/home12/cfpark00/venv1/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: rch is an invalid version and will not be supported in a future release
  warnings.warn(


In [4]:
from mltools.utils import cuda_tools
device=cuda_tools.get_freer_device()

memory_available [78291]
best GPU: 0


In [2]:
from mltools.models.vae_model import AutoencoderKL
enc_dec_params = dict(
    shape=(3,256,256),
    chs=[48, 96, 192],
    attn_sizes=[],
    mid_attn=False,
    num_res_blocks=1,
    dropout_prob=0.0,
    z_channels=4,
    double_z=True,
    n_attention_heads=1,
    norm_groups=8,
    norm_eps=1e-6,
    norm_affine=True,
    act="gelu",
    conv_kernel_size=3,
    conv_padding_mode="zeros",
)
vae=AutoencoderKL(enc_dec_params=enc_dec_params)
vae=vae.to(device)
x=torch.rand(1,3,256,256).to(device)
x_rec=vae.get_reconstuctions(x)
x.shape,x_rec.shape

(torch.Size([1, 3, 256, 256]), torch.Size([1, 3, 256, 256]))

In [26]:
from mltools.networks.networks import CUNet
shape=(3,256,256)
score_model=CUNet(
    shape=shape,
    chs=[48, 48, 192, 200],
    s_conditioning_channels=2,
    v_conditioning_dims = [3,6],
    v_embedding_dim = 64,
    t_conditioning=True,
    t_embedding_dim=64,
    mid_attn=True,
    dropout_prob=0.0,
    n_attention_heads=1,
    norm_groups=8,
    conv_padding_mode="zeros",
)
score_model=score_model.to(device)
batch_size=2
x=torch.randn(batch_size,*shape).to(device)
t=torch.randn(batch_size).to(device)
s=torch.randn(batch_size,2,*shape[1:]).to(device)
v=[torch.randn(batch_size,3).to(device),torch.randn(batch_size,6).to(device)]

In [27]:
res=score_model(x,t=t,s_conditioning=s,v_conditionings=v)
x.shape,res.shape

(torch.Size([2, 3, 256, 256]), torch.Size([2, 3, 256, 256]))

In [31]:
from mltools.models.vdm_model import VDM
vdm=VDM(
    score_model=score_model,
    noise_schedule= "fixed_linear",
    gamma_min = -13.3,
    gamma_max = 5.0,
    antithetic_time_sampling = True,
    image_shape = shape,
    data_noise = 1.0e-3,
)
vdm=vdm.to(device)

In [32]:
res=vdm.sample(batch_size=batch_size,conditioning=s,conditioning_values=v,n_sampling_steps=50,device=device)
res.shape

torch.Size([2, 3, 256, 256])

In [1]:
from mltools.utils import cuda_tools
device=cuda_tools.get_freer_device()
import torch

memory_available [78291]
best GPU: 0


## SFM

In [66]:
from mltools.utils import cuda_tools
device=cuda_tools.get_freer_device()
import torch

import mltools.models.sfm_model as sfm_model
import mltools.networks.networks as networks

memory_available [78995]
best GPU: 0


In [67]:
import importlib
importlib.reload(sfm_model)

<module 'mltools.models.sfm_model' from '/n/home12/cfpark00/MLtools/mltools/models/sfm_model.py'>

In [68]:
shape=(1,256,256)
conditioning_channels=1

In [69]:
net=networks.CUNet(shape=shape,s_conditioning_channels=1,t_conditioning=True)
net=net.to(device)

In [70]:
x0=torch.randn(1,*shape).to(device)
x1=torch.randn(1,*shape).to(device)

In [71]:
sfm=sfm_model.LightSFM(velocity_model=net)
sfm.sfm.compute_loss(x0=x0,x1=x1)

tensor(3.9954, device='cuda:0', grad_fn=<MeanBackward0>)

In [72]:
sfm.sfm.predict(x0,verbose=1).shape

Integrating...: 100%|██████████| 100/100 [00:00<00:00, 145.47it/s]


torch.Size([1, 1, 256, 256])

In [3]:
import torch
import importlib

In [4]:
from mltools.networks import blocks
from mltools.networks import networks
from mltools.models import gpt_model
from mltools.models import configs
importlib.reload(configs)
importlib.reload(blocks)
importlib.reload(networks)
importlib.reload(gpt_model)

<module 'mltools.models.gpt_model' from '/n/home12/cfpark00/MLtools/mltools/models/gpt_model.py'>

In [5]:
c=configs.GPTConfig(
    block_size=1024,
    tokenized=True,
    in_size=5000,
    n_layer= 4,
    n_head = 4,
    n_embd = 128,
    dropout = 0.0,
    bias = True,
    causal = True,
    verbose=0
)

In [6]:
transformer=networks.Transformer(c)

In [7]:
gpt=gpt_model.GPT(transformer)

In [8]:
gpt(torch.randint(0,256,(1,2))).shape

torch.Size([1, 2, 5000])

In [30]:
sa=blocks.SelfAttentionBlock(c)

In [32]:
sa(torch.randn(1,4,128)).shape

torch.Size([1, 4, 128])

In [10]:
c

GPTConfig(block_size=1024, vocab_size=50304, n_layer=12, n_head=12, n_embd=768, dropout=0.0, bias=True, causal=True)

In [5]:
x=torch.rand(1,3,256,256).type_as(torch.FloatTensor()).to(device)   
x

tensor([[[[0.9182, 0.1021, 0.3288,  ..., 0.8417, 0.3836, 0.4547],
          [0.7041, 0.4940, 0.6841,  ..., 0.4371, 0.2962, 0.3629],
          [0.9874, 0.6139, 0.5383,  ..., 0.0901, 0.6551, 0.0013],
          ...,
          [0.8798, 0.4551, 0.7846,  ..., 0.6416, 0.9386, 0.1934],
          [0.7632, 0.7251, 0.6380,  ..., 0.2853, 0.9538, 0.9870],
          [0.4061, 0.3605, 0.5263,  ..., 0.8869, 0.2065, 0.7958]],

         [[0.8947, 0.6882, 0.0060,  ..., 0.1786, 0.1728, 0.4353],
          [0.7298, 0.4708, 0.4322,  ..., 0.6245, 0.3530, 0.5798],
          [0.8705, 0.2484, 0.8823,  ..., 0.0433, 0.2844, 0.7832],
          ...,
          [0.1817, 0.2483, 0.7453,  ..., 0.5360, 0.2481, 0.2466],
          [0.1852, 0.2051, 0.8311,  ..., 0.1764, 0.5672, 0.7622],
          [0.8441, 0.7272, 0.6660,  ..., 0.2331, 0.3785, 0.2704]],

         [[0.3158, 0.0558, 0.7052,  ..., 0.3508, 0.2991, 0.4315],
          [0.8077, 0.9275, 0.0672,  ..., 0.3478, 0.3862, 0.0666],
          [0.6117, 0.1628, 0.0806,  ..., 0

In [6]:
x.dim()

4